In [ ]:
# Import FABlib 

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Slice / Node Params

slice_name = 'MySlice'

site = 'TACC'

node1_name = 'node1'
node2_name = 'node2'
node3_name = 'node3'
node4_name = 'node4'

network_name='net'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'
node4_nic_name = 'nic4'

In [ ]:
# Create slice and configure nodes

try:
    # Create slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node 1
    node1 = slice.add_node(name=node1_name, site=site, image='default_ubuntu_20', cores=4, ram=8, disk=16)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]

    # Node 2
    node2 = slice.add_node(name=node2_name, site=site, image='default_ubuntu_20', cores=4, ram=8, disk=16)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]

    # Node 3
    node3 = slice.add_node(name=node3_name, site=site, image='default_ubuntu_20', cores=4, ram=8, disk=16)
    iface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]

    # Node 4
    node4 = slice.add_node(name=node4_name, site=site, image='default_ubuntu_20', cores=4, ram=8, disk=16)
    iface4 = node4.add_component(model='NIC_Basic', name=node4_nic_name).get_interfaces()[0]
    
    # Network setup
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2, iface3, iface4], type='L2Bridge')
    
    slice.submit()
            
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Configure nodes and routes

try:
    slice = fablib.get_slice(slice_name)
    
    # Node 1
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name)  
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    node1.execute('sudo ip link set dev ens7 up')
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')    
    print("Node 1 IP:", node1_addr)
    
    # Node 2
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    node2.execute('sudo ip link set dev ens7 up')
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')    
    print("Node 2 IP:", node2_addr)
    
    # Node 3
    node3 = slice.get_node(name=node3_name)        
    node3_iface = node3.get_interface(network_name=network_name)  
    node3_addr = available_ips.pop(0)
    node3_iface.ip_addr_add(addr=node3_addr, subnet=subnet)
    node3.execute('sudo ip link set dev ens7 up')
    stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')    
    print("Node 3 IP:", node3_addr)
    
    # Node 4
    node4 = slice.get_node(name=node4_name)        
    node4_iface = node4.get_interface(network_name=network_name)  
    node4_addr = available_ips.pop(0)
    node4_iface.ip_addr_add(addr=node4_addr, subnet=subnet)
    node4.execute('sudo ip link set dev ens7 up')
    stdout, stderr = node4.execute(f'ip addr show {node4_iface.get_os_interface()}')    
    print("Node 4 IP:", node4_addr)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Setup VM environment

from ipaddress import ip_address, IPv6Address

nodes      = [node1, node2, node3, node4]
node_addrs = [node1_addr, node2_addr, node3_addr, node4_addr]
addrs_str = f'{node1_addr} {node2_addr} {node3_addr} {node4_addr}'

try:   
    for node, addr in zip(nodes, node_addrs):
        node_name = node.get_name()
        other_nodes_addr = ' '.join([str(x) for x in node_addrs if x != addr])
        
        if type(ip_address(node.get_management_ip())) is IPv6Address: # ipv6 fix for github clone
            node.upload_file('nat64.sh', 'nat64.sh')
            stdout, stderr = node.execute(f'chmod +x nat64.sh && ./nat64.sh')
            
        result = node.upload_file('./fix/svs_base.py','svs_base.py')
        result = node.upload_file('./fix/svs_thread.py', 'svs_thread.py')
        result = node.upload_file('./fix/svs_base_thread.py', 'svs_base_thread.py')
        result = node.upload_file('./fix/svs_shared_thread.py', 'svs_shared_thread.py')
        result = node.upload_file('client.sh','client.sh')
        
        result = node.upload_file('node_config.sh','node_config.sh')
        stdout, stderr = node.execute(f'chmod +x node_config.sh && ./node_config.sh {node_name} {addr} {addrs_str} &> config.log')
        
        result = node.upload_file('nginx_setup.sh','nginx_setup.sh')
        stdout, stderr = node.execute(f'chmod +x nginx_setup.sh && ./nginx_setup.sh {node_name}')
        
        print(f"{node.get_name()} SSH Command: {node.get_ssh_command()}")
        
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# # Delete slice

# try:
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")